In [1]:
import pandas as pd
import string
import re
import pymongo
#from pymongo import MongoClient 
from nltk.tokenize import TweetTokenizer 

client = pymongo.MongoClient('mongodb://TwitterIPO?authSource=TwitterIPO')

client.TwitterIPO["RawTweetSentiments"].delete_many({})
def clean_tweet(t):

    t = t.replace('RT ','')
    t = t.replace('$','')
    t = t.replace('#','')
    
    for word in t:
        if word.startswith(("@")) == True:
            t = t.replace(word,'')
            
    t = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''','',t)

    t = t.replace(',','')
    t = t.replace('.','')
    t = t.replace(';','')
    t = t.replace("'",'')
    t = t.replace('"','')
    t = t.replace('...','')
    
    return(t.lower())

def binarySearchOfCharacterInWord(alphabetDictionary,twoDimensionalArray, item): 
    first = 0
    last = len(twoDimensionalArray)-1
    found = False
    charIndex=1
    midpoint=-1
    dontContinue=False
  
    while first<=last and charIndex < len(item) and  dontContinue==False :
        midpoint = (first + last)//2
        currentWordFromDictionary=twoDimensionalArray[midpoint]

        
        while(len(currentWordFromDictionary)<len(item)):
            midpoint=midpoint+1
            currentWordFromDictionary=twoDimensionalArray[midpoint]
            
        while(charIndex < len(item) and currentWordFromDictionary!=item and currentWordFromDictionary[charIndex] == item[charIndex]):
            charIndex=charIndex+1
            if(currentWordFromDictionary==item):
                found = True
                
        if(charIndex==len(item)):
            charIndex=charIndex-1
            
        if(len(currentWordFromDictionary)>len(item) and (charIndex+1)==len(item) and item[:charIndex]==twoDimensionalArray[midpoint][:charIndex]):
            dontContinue=True
            
    
        print(currentWordFromDictionary)
        print(item)
        print(charIndex)
        if dontContinue==False and not found and len(currentWordFromDictionary)>=len(item) and currentWordFromDictionary[charIndex] != item[charIndex]:
            currentcharacter=item[charIndex]
            currentAlistCharacter=twoDimensionalArray[midpoint][charIndex]
           
            previousChars=item[:charIndex]
            previousCharsInDict=twoDimensionalArray[midpoint][:charIndex]
            if(previousChars!=previousCharsInDict):
                if(alphabetDictionary[previousChars[-1:]] < alphabetDictionary[previousCharsInDict[-1:]]):
                    last = midpoint-1
                else:
                    first = midpoint+1
            else:
                if alphabetDictionary[currentcharacter] < alphabetDictionary[currentAlistCharacter]:
                    last = midpoint-1
                else:
                    first = midpoint+1
        
    if(found):
        print("Found")
    else:
        midpoint=-1
        print("Not Found")
    return midpoint

listofAlphabet=list(string.ascii_lowercase)
alphabetDictionary= {}
alphabetIntegerValue=1
for alphabet in listofAlphabet:
    alphabetDictionary[alphabet]=alphabetIntegerValue
    alphabetIntegerValue=alphabetIntegerValue+1


postiveWordArray=[]
negativeWordArray=[]
positiveWord= client.TwitterIPO.PositiveWords.find({},{"Word":1}).sort([("Word",pymongo.ASCENDING)])
negativeWord= client.TwitterIPO.NegativeWords.find({},{"Word":1}).sort([("Word",pymongo.ASCENDING)])
positiveWordDF =  pd.DataFrame(list(positiveWord))
negativeWordDF =  pd.DataFrame(list(negativeWord))

for alphabet in listofAlphabet:
    index=alphabetDictionary[alphabet]-1
    alphabetPositiveWords=positiveWordDF[positiveWordDF[positiveWordDF.columns[0]].str.startswith(alphabet.lower())]
    alphabetNegativeWords=negativeWordDF[negativeWordDF[negativeWordDF.columns[0]].str.startswith(alphabet.lower())]
    postiveWordArray.append(alphabetPositiveWords[alphabetPositiveWords.columns[0]].str.lower().values)
    negativeWordArray.append(alphabetNegativeWords[alphabetNegativeWords.columns[0]].str.lower().values)


    
cursor=client.TwitterIPO.RawTweets.find({ "lang": "en" } )
tweetDataFrame =  pd.DataFrame(list(cursor))
print("Start")
tweetmessages=pd.unique(tweetDataFrame[['tweet_id', 'text']].values)
tknz = TweetTokenizer()
for tweet in tweetmessages:
    cleantweet= clean_tweet(tweet[1])
    tokenizetweet= tknz.tokenize(cleantweet)
    numberOfPositiveWords=0
    numberOfNegativeWords=0
    for wordInTweet in tokenizetweet:
        alphabetInWord=wordInTweet[0]
        if alphabetInWord in alphabetDictionary:
            print(alphabetInWord)
            alphabetIndex=alphabetDictionary[alphabetInWord]-1
            posArray=postiveWordArray[alphabetIndex]
            negArray=negativeWordArray[alphabetIndex]
            for posword in posArray:
                if(posword==wordInTweet):
                    print("positive word : ",wordInTweet)
                    numberOfPositiveWords=numberOfPositiveWords+1
                    break
            for negword in negArray:
                if(negword==wordInTweet):
                    print("negative word : ",wordInTweet)
                    numberOfNegativeWords=numberOfNegativeWords+1
                    break
            #indexOfPositive=binarySearchOfCharacterInWord(alphabetDictionary,posDict,wordInTweet)
    sentimentpostivenegative = {"tweet_id": tweet[0],"number_positive_words": numberOfPositiveWords,"number_negative_words": numberOfNegativeWords, "total_number_of_words":len(tokenizetweet)}
    mongotweetid= client.TwitterIPO["RawTweetSentiments"].insert_one(sentimentpostivenegative).inserted_id
  



Start
positive word :  worth
positive word :  powerful
positive word :  blasted
negative word :  blasted
positive word :  perfect
positive word :  plus
negative word :  unusually
positive word :  strong
negative word :  strong
negative word :  new
negative word :  new
positive word :  gold
positive word :  leading
positive word :  winner
negative word :  short
negative word :  base
negative word :  new
positive word :  opportunity
negative word :  gone
positive word :  hot
negative word :  new
positive word :  top
positive word :  nice
positive word :  top
positive word :  winner
negative word :  last
positive word :  leads
negative word :  could
positive word :  double
negative word :  smoke
positive word :  partial
positive word :  gold
positive word :  higher
positive word :  led
negative word :  breaks
positive word :  recover
negative word :  breaks
positive word :  received
positive word :  quiet
negative word :  quiet
negative word :  expire
negative word :  last
negative word :